# Pipeline SDK

In [1]:
import kfp

@kfp.dsl.pipeline(name='my-pipeline_1207-0027', description='Hello How do you do')
def hello_pipeline():
    train_task = hello_component()

@kfp.dsl.component
def hello_component():
    return kfp.dsl.ContainerOp(
        name='my-ContainerOp',
        #image='kangwoo/kfp-mnist:kfp' # TODO 본 교육용으로 바꿀 것
        image='registry.kube-system.svc.cluster.local:30000/covid19-katib-job:latest'
    )    

# Pipeline 컴파일

In [2]:
kfp.compiler.Compiler().compile(hello_pipeline, 'my_pipeline.zip')

/home/jovyan/.local/lib/python3.6/site-packages/kfp/dsl/_container_op.py:1039: FutureWarning: Please create reusable components instead of constructing ContainerOp instances directly. Reusable components are shareable, portable and have compatibility and support guarantees. Please see the documentation: https://www.kubeflow.org/docs/pipelines/sdk/component-development/#writing-your-component-definition-file The components can be created manually (or, in case of python, using kfp.components.create_component_from_func or func_to_container_op) and then loaded using kfp.components.load_component_from_file, load_component_from_uri or load_component_from_text: https://kubeflow-pipelines.readthedocs.io/en/stable/source/kfp.components.html#kfp.components.load_component_from_file
  category=FutureWarning,


In [3]:
! ls -l my_pipeline.zip

-rw-r--r-- 1 jovyan users 545 Mar 15 09:21 my_pipeline.zip


In [4]:
! unzip -l my_pipeline.zip

Archive:  my_pipeline.zip
  Length      Date    Time    Name
---------  ---------- -----   ----
      882  1980-01-01 00:00   pipeline.yaml
---------                     -------
      882                     1 file


In [5]:
! unzip -p my_pipeline.zip

apiVersion: argoproj.io/v1alpha1
kind: Workflow
metadata:
  generateName: my-pipeline-1207-0027-
  annotations: {pipelines.kubeflow.org/kfp_sdk_version: 1.4.0, pipelines.kubeflow.org/pipeline_compilation_time: '2021-03-15T09:21:09.998507',
    pipelines.kubeflow.org/pipeline_spec: '{"description": "Hello How do you do",
      "name": "my-pipeline_1207-0027"}'}
  labels: {pipelines.kubeflow.org/kfp_sdk_version: 1.4.0}
spec:
  entrypoint: my-pipeline-1207-0027
  templates:
  - name: my-containerop
    container: {image: 'registry.kube-system.svc.cluster.local:30000/covid19-katib-job:latest'}
    metadata:
      annotations: {pipelines.kubeflow.org/component_spec: '{"name": "Hello component"}'}
  - name: my-pipeline-1207-0027
    dag:
      tasks:
      - {name: my-containerop, template: my-containerop}
  arguments:
    parameters: []
  serviceAccountName: pipeline-runner


# Pipeline Experiment

In [6]:
client = kfp.Client()

my_experiment = client.create_experiment(
    name='My first Experiment' # Experiment name
)

ApiException: (403)
Reason: Forbidden
HTTP response headers: HTTPHeaderDict({'content-length': '19', 'content-type': 'text/plain', 'date': 'Mon, 15 Mar 2021 09:21:11 GMT', 'server': 'envoy', 'x-envoy-upstream-service-time': '0'})
HTTP response body: RBAC: access denied


# Pipeline 실행

In [ ]:
my_run = client.run_pipeline(
    my_experiment.id, 
    'oh_my_experiment', # Run name
    'my_pipeline.zip'
)